In [46]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [47]:
import torch.nn.functional as F
import math

from torch.optim import lr_scheduler

In [48]:
import argparse
import torch.optim as optim
from torchvision import datasets, transforms

In [64]:

# MNIST Dataset
train_dataset = dsets.MNIST(root='./data/',
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data/',
                           train=False, 
                           transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)

In [50]:
def squash(x):
    lengths2 = x.pow(2).sum(dim=2)
    lengths = lengths2.sqrt()
    x = x * (lengths2 / (1 + lengths2) / lengths).view(x.size(0), x.size(1), 1)
    return x

In [51]:
class Encode_layer(nn.Module):
    def __init__(self, input_dim, output_dim, output_caps):
        super(Encode_layer, self).__init__()
        
        self.layer = nn.Linear(input_dim, output_dim*output_caps)
        self.output_dim = output_dim
        self.output_caps = output_caps
        
    def forward(self, input) : 
        output = self.layer(input)
        output = F.relu(output)
        output = output.view(output.shape[0], self.output_caps, self.output_dim)
        output = squash(output)
        
        return output

In [52]:
class Neuron_layer(nn.Module):
    def __init__(self, input_neurons, input_dim, output_dim, output_neurons):
        super(Neuron_layer, self).__init__()
        self.input_neurons = input_neurons
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.output_neurons = output_neurons
        self.weights = nn.Parameter(torch.Tensor(input_neurons, input_dim, output_neurons * output_dim))
        self.reset_parameters()
        
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.input_neurons)
        self.weights.data.uniform_(-stdv, stdv)
        
    def forward(self, input):
        
        input = input.unsqueeze(2)
        predict = input.matmul(self.weights)
        predict = predict.view(predict.size(0), self.input_neurons, self.output_neurons, self.output_dim)
        
        return predict
        

In [53]:
map_shape = [4, 3, 10]
leng = 3
bs = []
batch_size=100

for (i, j) in zip(map_shape[:-1], map_shape[1:]):
    bs.append(torch.zeros((batch_size, i,j)))
        
def backprop(reward) :
        
    bs[1]=reward[1].data
    
    for k in range(batch_size):
        for state in range(bs[0].shape[1]) :
            max_value, _ = torch.max(bs[1][k], dim=1)
            bs[0][k][state] =reward[0].data[k][state] + max_value
                

In [54]:
def train(epoch):
    model.train()
    correct=0.
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data.view(data.shape[0], -1)), Variable(target, requires_grad=False)
        optimizer.zero_grad()
        probs, reward = model(data, bs)
        print (probs[0])
        
        backprop(reward)
        
        loss = loss_fn(probs, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.data[0]))
            
        pred = probs.data.max(1, keepdim=True)[1]  # get the index of the max probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    print('Accuracy : {:.2f}%'.format(100. * correct / len(train_loader.dataset)))

In [61]:
len(bs)

2

In [62]:
bs[0].shape

torch.Size([100, 4, 3])

In [63]:
bs[1].shape

torch.Size([100, 3, 10])

In [66]:
#Test
total=0
correct=0.
#batch size 다를 땐 테스트가 안 됌?
for images, labels in test_loader:
    images = Variable(images.view(images.shape[0], -1))
    outputs, rewards = model(images, bs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))


Test Accuracy of the model on the 10000 test images: 55 %


In [68]:
adv_test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=1, 
                                          shuffle=False)

In [72]:
dummy=1
s=0.
t=0.

test_b0=bs[0][0].unsqueeze(0)
test_b1=bs[1][0].unsqueeze(0)

test_bs=[test_b0, test_b1]


for (x, y) in adv_test_loader :
    x, y = Variable(x, requires_grad=True), Variable(y, requires_grad=False)
    probs, _ = model(x.view(x.shape[0], -1), test_bs)
    y_pred = np.argmax(probs.data.numpy())
    loss = nn.CrossEntropyLoss()(probs, y)
    loss.backward()
    epsilon = 0.1 
    x_grad   = torch.sign(x.grad.data)
    x_adversarial = torch.clamp(x.data + epsilon * x_grad, 0, 1)
    adversarial_probs, _ = model(Variable(x_adversarial).view(x.shape[0], -1), test_bs)
    y_pred_adversarial = np.argmax(adversarial_probs.data.numpy())
    
    print("{0} th example ".format(dummy))
    print ("True value: "+ str(y.data.numpy()[0])+"\nPredicted value : "+str(y_pred)+ "\nAdversarial :" + str(y_pred_adversarial)+"\n" )
    
    dummy+=1
    
    if y.data.numpy()[0]!=y_pred :
        t+=1
    
    if y_pred!=y_pred_adversarial :
        s+=1
    if dummy==100:
        break
        
print("Accuracy of test_model : {0:.3f} , Adversarials : {1:.3f}".format((t/dummy)*100, (s/dummy)*100))

1 th example 
True value: 7
Predicted value : 7
Adversarial :7

2 th example 
True value: 2
Predicted value : 2
Adversarial :2

3 th example 
True value: 1
Predicted value : 1
Adversarial :3

4 th example 
True value: 0
Predicted value : 0
Adversarial :0

5 th example 
True value: 4
Predicted value : 4
Adversarial :9

6 th example 
True value: 1
Predicted value : 1
Adversarial :7

7 th example 
True value: 4
Predicted value : 4
Adversarial :8

8 th example 
True value: 9
Predicted value : 9
Adversarial :3

9 th example 
True value: 5
Predicted value : 5
Adversarial :6

10 th example 
True value: 9
Predicted value : 9
Adversarial :7

11 th example 
True value: 0
Predicted value : 0
Adversarial :5

12 th example 
True value: 6
Predicted value : 6
Adversarial :2

13 th example 
True value: 9
Predicted value : 9
Adversarial :2

14 th example 
True value: 0
Predicted value : 0
Adversarial :0

15 th example 
True value: 1
Predicted value : 1
Adversarial :3

16 th example 
True value: 5
Predi

In [55]:
class Learning(nn.Module) :
    def __init__(self) :
        super(Learning, self).__init__()
        
        self.network1 = Encode_layer(784, 128, 4)
        self.network2 = Neuron_layer(4, 128, 32, 3)
        self.network3 = Neuron_layer(3, 32, 16, 10)
        
    def forward(self, input, bs):
        
        self.bs=bs
        output1 = self.network1(input)
        output2 = self.network2(output1)
        
        c2=F.softmax(nn.Parameter(self.bs[0].view(-1, self.bs[0].shape[2])), dim=1).view(-1, self.bs[0].shape[1], self.bs[0].shape[2], 1)
        
        s2 = (c2*output2).sum(dim=1)
        v2 = squash(s2)
        
        v2_hat = v2.unsqueeze(1)
        reward0 = torch.squeeze((output2 * v2_hat).sum(-1))
        
        output3 = self.network3(v2)
        
        c3=F.softmax(nn.Parameter(self.bs[1].view(-1, self.bs[1].shape[2])), dim=1).view(-1, self.bs[1].shape[1], self.bs[1].shape[2], 1)
        s3 = (c3*output3).sum(dim=1)
        v3 = squash(s3)
        
        v3_hat = v3.unsqueeze(1)
        reward1 = torch.squeeze((output3 * v3_hat).sum(-1))
        reward = [reward0, reward1]
        
        probs = v3.pow(2).sum(dim=2).sqrt()
        
        return probs, reward

In [56]:
model=Learning()

In [57]:
class MarginLoss(nn.Module):
    def __init__(self, m_pos, m_neg, lambda_):
        super(MarginLoss, self).__init__()
        self.m_pos = m_pos
        self.m_neg = m_neg
        self.lambda_ = lambda_

    def forward(self, lengths, targets, size_average=True):
        t = torch.zeros(lengths.size()).long()
        if targets.is_cuda:
            t = t.cuda()
        t = t.scatter_(1, targets.data.view(-1, 1), 1)
        targets = Variable(t)
        losses = targets.float() * F.relu(self.m_pos - lengths).pow(2) + \
                 self.lambda_ * (1. - targets.float()) * F.relu(lengths - self.m_neg).pow(2)
        return losses.mean() if size_average else losses.sum()


In [58]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)

scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=15, min_lr=1e-6)

loss_fn = MarginLoss(0.9, 0.1, 0.5)

In [59]:
import numpy as np

In [60]:
for epoch in range(10) :
    print(epoch)
    train(epoch)

0
Variable containing:
1.00000e-02 *
  0.6766
  0.3558
  0.5954
  0.4880
  1.2675
  1.4465
  0.3738
  0.3754
  1.1180
  0.9845
[torch.FloatTensor of size 10]

Train Epoch: 0 [0/60000 (0%)]	Loss: 0.079957
Variable containing:
1.00000e-02 *
  1.2095
  0.6790
  0.9429
  0.4304
  1.8373
  1.0554
  1.3667
  0.7583
  0.5570
  0.9908
[torch.FloatTensor of size 10]

Variable containing:
1.00000e-02 *
  0.9897
  1.3325
  0.9131
  0.7436
  1.6582
  0.9015
  1.1112
  1.1296
  0.8337
  0.7301
[torch.FloatTensor of size 10]

Variable containing:
1.00000e-02 *
  2.1706
  0.9534
  1.0818
  0.9982
  1.8916
  0.8408
  1.7994
  1.5836
  1.5604
  2.3968
[torch.FloatTensor of size 10]

Variable containing:
1.00000e-03 *
  5.8040
  2.1020
  3.1855
  4.7375
  7.2805
  7.8074
  8.0137
  5.6882
  8.1509
  6.8839
[torch.FloatTensor of size 10]

Variable containing:
1.00000e-02 *
  2.4857
  1.3228
  1.4220
  0.6193
  2.5437
  1.6338
  2.1542
  1.5544
  1.7233
  2.5027
[torch.FloatTensor of size 10]

Variable co

Variable containing:
 0.0799
 0.0113
 0.0422
 0.0392
 0.0966
 0.0573
 0.0592
 0.0798
 0.0915
 0.1218
[torch.FloatTensor of size 10]

Variable containing:
 0.0911
 0.0167
 0.0509
 0.0226
 0.1103
 0.0359
 0.1193
 0.0617
 0.0980
 0.1242
[torch.FloatTensor of size 10]

Variable containing:
 0.1505
 0.0189
 0.0587
 0.0296
 0.0822
 0.0474
 0.0953
 0.0602
 0.0868
 0.1068
[torch.FloatTensor of size 10]

Variable containing:
 0.1104
 0.0132
 0.0456
 0.0283
 0.1075
 0.0395
 0.0835
 0.0664
 0.1087
 0.1238
[torch.FloatTensor of size 10]

Variable containing:
 0.0599
 0.0113
 0.0338
 0.0278
 0.1029
 0.0401
 0.0526
 0.0611
 0.0815
 0.1223
[torch.FloatTensor of size 10]

Variable containing:
 0.0674
 0.0394
 0.0431
 0.0286
 0.0764
 0.0449
 0.0671
 0.0792
 0.1135
 0.1167
[torch.FloatTensor of size 10]

Variable containing:
1.00000e-02 *
  4.5123
  4.3661
  2.7026
  2.1554
  4.5545
  4.4713
  3.0780
  7.3206
  7.1209
  8.1784
[torch.FloatTensor of size 10]

Variable containing:
 0.0680
 0.0232
 0.0504


Variable containing:
 0.1029
 0.0225
 0.1125
 0.0471
 0.0937
 0.0845
 0.1661
 0.0282
 0.1080
 0.0753
[torch.FloatTensor of size 10]

Variable containing:
 0.0773
 0.0531
 0.0831
 0.0284
 0.1060
 0.0755
 0.0896
 0.0631
 0.1211
 0.1273
[torch.FloatTensor of size 10]

Variable containing:
 0.0480
 0.0169
 0.0716
 0.0222
 0.1124
 0.0604
 0.1710
 0.0380
 0.0861
 0.0973
[torch.FloatTensor of size 10]

Variable containing:
 0.0596
 0.0302
 0.0435
 0.0269
 0.1474
 0.0534
 0.0789
 0.1053
 0.1120
 0.1708
[torch.FloatTensor of size 10]

Variable containing:
 0.0522
 0.0342
 0.0499
 0.0603
 0.1200
 0.0594
 0.0384
 0.1513
 0.0973
 0.1698
[torch.FloatTensor of size 10]

Variable containing:
 0.0733
 0.0237
 0.1061
 0.0897
 0.1235
 0.1004
 0.0781
 0.0689
 0.1158
 0.1293
[torch.FloatTensor of size 10]

Variable containing:
 0.0910
 0.0328
 0.0374
 0.0382
 0.0926
 0.0737
 0.0369
 0.1544
 0.0848
 0.1287
[torch.FloatTensor of size 10]

Variable containing:
 0.1110
 0.0458
 0.0761
 0.0735
 0.0673
 0.1369


Variable containing:
 0.0551
 0.0400
 0.0586
 0.1722
 0.0633
 0.1295
 0.0273
 0.0935
 0.1133
 0.0953
[torch.FloatTensor of size 10]

Variable containing:
 0.1233
 0.0524
 0.1127
 0.2022
 0.0420
 0.1907
 0.0486
 0.0450
 0.0845
 0.0531
[torch.FloatTensor of size 10]

Variable containing:
 0.0970
 0.0436
 0.1340
 0.1851
 0.0726
 0.2154
 0.0593
 0.0513
 0.1425
 0.0709
[torch.FloatTensor of size 10]

Variable containing:
 0.0654
 0.0277
 0.0311
 0.0590
 0.1466
 0.1201
 0.0325
 0.1494
 0.1292
 0.1867
[torch.FloatTensor of size 10]

Variable containing:
 0.0858
 0.0230
 0.1428
 0.0362
 0.1218
 0.0860
 0.1718
 0.0259
 0.0877
 0.0762
[torch.FloatTensor of size 10]

Variable containing:
 0.0685
 0.0379
 0.0997
 0.0615
 0.1201
 0.0986
 0.0830
 0.0802
 0.2122
 0.1262
[torch.FloatTensor of size 10]

Variable containing:
 0.0818
 0.0351
 0.0736
 0.1381
 0.0663
 0.1547
 0.0539
 0.0423
 0.1420
 0.0818
[torch.FloatTensor of size 10]

Train Epoch: 0 [18000/60000 (30%)]	Loss: 0.051619
Variable containing

Variable containing:
 0.1039
 0.1069
 0.0885
 0.0878
 0.0841
 0.1669
 0.0632
 0.1163
 0.1575
 0.1108
[torch.FloatTensor of size 10]

Variable containing:
 0.0554
 0.0436
 0.0248
 0.0705
 0.0998
 0.1003
 0.0149
 0.2519
 0.0979
 0.1616
[torch.FloatTensor of size 10]

Variable containing:
 0.1206
 0.0208
 0.0633
 0.0976
 0.1102
 0.1343
 0.0540
 0.2147
 0.1714
 0.1675
[torch.FloatTensor of size 10]

Variable containing:
 0.0618
 0.0326
 0.0262
 0.0666
 0.0655
 0.0727
 0.0094
 0.3159
 0.0757
 0.1446
[torch.FloatTensor of size 10]

Variable containing:
 0.0263
 0.1831
 0.0390
 0.0261
 0.0203
 0.0560
 0.0159
 0.2483
 0.0352
 0.0635
[torch.FloatTensor of size 10]

Train Epoch: 0 [24000/60000 (40%)]	Loss: 0.047181
Variable containing:
 0.0723
 0.2234
 0.0729
 0.0521
 0.0476
 0.1057
 0.0571
 0.1387
 0.1641
 0.0903
[torch.FloatTensor of size 10]

Variable containing:
 0.0564
 0.0741
 0.0474
 0.1430
 0.0951
 0.1115
 0.0216
 0.1186
 0.1447
 0.1602
[torch.FloatTensor of size 10]

Variable containing

...

Variable containing:
 0.0008
 0.0011
 0.0005
 0.0024
 0.8450
 0.0206
 0.0249
 0.0048
 0.0398
 0.5352
[torch.FloatTensor of size 10]

Variable containing:
 0.0001
 0.0863
 0.0001
 0.0001
 0.0328
 0.0001
 0.0002
 0.0001
 0.0002
 0.1622
[torch.FloatTensor of size 10]

Variable containing:
 0.0024
 0.0097
 0.0039
 0.0042
 0.0040
 0.0044
 0.0026
 0.6538
 0.0059
 0.0172
[torch.FloatTensor of size 10]

Variable containing:
 1.2682e-05
 1.3018e-02
 1.5025e-05
 3.5173e-05
 2.3087e-05
 1.8052e-05
 9.9684e-06
 2.1196e-01
 2.5626e-05
 2.3780e-05
[torch.FloatTensor of size 10]

Variable containing:
 0.0226
 0.0018
 0.6967
 0.0047
 0.0066
 0.0057
 0.0049
 0.0031
 0.0062
 0.0042
[torch.FloatTensor of size 10]

Variable containing:
 1.2006e-05
 9.9937e-06
 3.6467e-01
 1.6275e-05
 2.3401e-05
 1.1954e-05
 9.0158e-03
 1.1339e-05
 1.5122e-05
 1.9756e-05
[torch.FloatTensor of size 10]

Variable containing:
 0.0030
 0.0031
 0.0039
 0.0042
 0.3613
 0.0021
 0.0033
 0.0427
 0.0145
 0.6212
[torch.FloatTensor of

Variable containing:
 0.0000
 0.0000
 0.0000
 0.0000
 0.9857
 0.0000
 0.0002
 0.0000
 0.0017
 0.0412
[torch.FloatTensor of size 10]

Variable containing:
 0.0000
 0.0000
 0.0001
 0.0001
 0.2859
 0.0001
 0.0224
 0.0000
 0.0000
 0.0001
[torch.FloatTensor of size 10]

Train Epoch: 9 [38000/60000 (63%)]	Loss: 0.047865
Variable containing:
 0.0071
 0.0063
 0.0069
 0.0139
 0.1766
 0.0064
 0.0038
 0.1684
 0.0170
 0.6279
[torch.FloatTensor of size 10]

Variable containing:
 0.0001
 0.0001
 0.0002
 0.0202
 0.0002
 0.0261
 0.0001
 0.0005
 0.0005
 0.2927
[torch.FloatTensor of size 10]

Variable containing:
 0.0054
 0.0032
 0.0027
 0.0352
 0.0161
 0.7200
 0.0198
 0.0061
 0.0138
 0.0168
[torch.FloatTensor of size 10]

Variable containing:
 0.0000
 0.0000
 0.0001
 0.0003
 0.0000
 0.9945
 0.0000
 0.0000
 0.0008
 0.0000
[torch.FloatTensor of size 10]

Variable containing:
 1.4646e-04
 2.3108e-07
 1.8658e-06
 3.4696e-07
 3.3205e-07
 2.4407e-01
 3.2554e-07
 3.0315e-07
 1.9985e-06
 3.1953e-07
[torch.Floa

Variable containing:
 0.0000
 0.0208
 0.0001
 0.0000
 0.0000
 0.2263
 0.0000
 0.0000
 0.0001
 0.0000
[torch.FloatTensor of size 10]

Variable containing:
 0.5128
 0.0015
 0.5673
 0.0221
 0.0119
 0.0054
 0.0023
 0.0048
 0.0036
 0.0016
[torch.FloatTensor of size 10]

Variable containing:
 0.0352
 0.0001
 0.1617
 0.0002
 0.0002
 0.0003
 0.0002
 0.0919
 0.0004
 0.0009
[torch.FloatTensor of size 10]

Train Epoch: 9 [44000/60000 (73%)]	Loss: 0.037562
Variable containing:
 0.0019
 0.0038
 0.0051
 0.0548
 0.0072
 0.0048
 0.0034
 0.8713
 0.0093
 0.0279
[torch.FloatTensor of size 10]

Variable containing:
 1.6760e-06
 7.3885e-04
 1.4475e-06
 1.5355e-06
 2.7649e-06
 4.2207e-06
 6.0137e-07
 1.9978e-01
 9.5946e-05
 1.2595e-06
[torch.FloatTensor of size 10]

Variable containing:
 0.0088
 0.0832
 0.0087
 0.0412
 0.0050
 0.6638
 0.0053
 0.0029
 0.0708
 0.0114
[torch.FloatTensor of size 10]

Variable containing:
 0.0001
 0.0001
 0.0001
 0.0001
 0.0177
 0.2998
 0.0000
 0.0001
 0.0001
 0.0022
[torch.Floa

Variable containing:
 0.0081
 0.0027
 0.0151
 0.0103
 0.0032
 0.6211
 0.0063
 0.0035
 0.0676
 0.0097
[torch.FloatTensor of size 10]

Variable containing:
 0.0001
 0.0001
 0.0001
 0.0001
 0.0273
 0.2292
 0.0000
 0.0001
 0.0001
 0.0060
[torch.FloatTensor of size 10]

Variable containing:
 0.0033
 0.0017
 0.0070
 0.0033
 0.0037
 0.0022
 0.7261
 0.0029
 0.0024
 0.0037
[torch.FloatTensor of size 10]

Variable containing:
 4.4198e-06
 4.4144e-06
 1.0572e-05
 6.6636e-06
 5.4382e-06
 4.4332e-06
 9.9569e-01
 4.1470e-06
 4.6230e-06
 7.8212e-06
[torch.FloatTensor of size 10]

Variable containing:
 6.1718e-08
 3.5955e-08
 1.3328e-07
 7.5742e-08
 3.2776e-05
 5.7995e-08
 2.4139e-01
 7.5595e-08
 1.8891e-07
 9.0143e-06
[torch.FloatTensor of size 10]

Variable containing:
 0.0025
 0.7385
 0.0051
 0.0015
 0.0025
 0.0020
 0.0022
 0.0032
 0.0043
 0.0026
[torch.FloatTensor of size 10]

Variable containing:
 2.1830e-04
 1.5893e-01
 8.4341e-06
 5.1763e-06
 4.7919e-06
 3.4738e-04
 1.3149e-04
 1.3210e-06
 3.25

Train Epoch: 9 [55000/60000 (92%)]	Loss: 0.047564
Variable containing:
 6.8314e-05
 8.9120e-06
 9.9498e-01
 1.6213e-05
 1.8771e-05
 1.2296e-05
 1.0663e-05
 8.8981e-06
 1.5108e-05
 1.3706e-05
[torch.FloatTensor of size 10]

Variable containing:
 3.8482e-06
 1.0867e-06
 8.3999e-01
 5.5692e-07
 4.1163e-06
 3.4630e-06
 4.9706e-05
 6.3423e-06
 3.3750e-07
 3.4028e-07
[torch.FloatTensor of size 10]

Variable containing:
 0.0011
 0.0006
 0.0103
 0.0021
 0.0016
 0.0013
 0.0006
 0.7381
 0.0041
 0.0068
[torch.FloatTensor of size 10]

Variable containing:
 3.7515e-03
 3.7971e-06
 3.5886e-05
 5.9919e-06
 5.3420e-06
 6.4375e-06
 4.8184e-06
 2.5469e-01
 1.6156e-05
 5.2828e-06
[torch.FloatTensor of size 10]

Variable containing:
 0.0061
 0.0022
 0.0048
 0.1013
 0.0041
 0.6897
 0.0048
 0.0039
 0.0173
 0.0110
[torch.FloatTensor of size 10]

Variable containing:
 0.0085
 0.0000
 0.0001
 0.0000
 0.0000
 0.4204
 0.0000
 0.0000
 0.0000
 0.0000
[torch.FloatTensor of size 10]

Variable containing:
 0.7254
 0.